In [1]:
using MathProgBase, Cbc

┌ Info: Precompiling Cbc [9961bab8-2fa3-5c5a-9d89-47fab24efd76]
└ @ Base loading.jl:1273


In [2]:
#function

struct list{T<:Integer}
    item::String
    weight::T
    value::T
    quant::T
end
Base.show(io::IO, kdps::list) = print(io, kdps.quant, " ", kdps.item, " ($(kdps.weight) kg, $(kdps.value) €)")
 
function solve(num::Vector{list{T}}, capacity::Integer) where T<:Integer
    w = getfield.(num, :weight)
    v = getfield.(num, :value)
    q = getfield.(num, :quant)
    
    sol = mixintprog(-v, w', '<', capacity, :Int, 0, q, CbcSolver())
    sol.status == :Optimal || error("this problem could not be solved")
 
    if all(q .== 1) # simpler case
        return num[sol.sol == 1.0]
    else
        pack = similar(num, 0)
        s = round.(Int, sol.sol)
        for (i, g) in enumerate(num)
            iszero(s[i]) && continue
            push!(pack, list(g.item, g.weight, g.value, s[i]))
        end
        return pack
    end
end

solve (generic function with 1 method)

In [ ]:
#main class


#Scenerio: A student wants to travel, she makes a list of what she wants to bring for the trip, she wish to park all 15 items,
           # but the total weight of all items is too much, She adds a value to each item, The value represents how important
           # the item is for trip.
           
         
#consider having a list of 15 items
#num variable consist of the list of items as follows:(Name of the item,weight,value,unit)

num = [list("T-shirt", 65, 150, 3),
        list("Dress", 70, 200, 2),
        list("water",100 , 130, 1),
        list("Trouder", 80, 180, 2),
        list("Ball", 15, 45, 1),
        list("Watch", 25, 60, 1),
        list("towel", 40, 50, 1),
        list("comb",5 , 55, 1),
        list("camera", 33, 70, 1),
        list("Mirror", 8, 65, 1),
        list("Pc", 95, 300, 1),
        list("Phone", 35, 250, 1),
        list("Book", 20, 80, 1),
        list("Earphones", 3, 75, 1),
        list("blanket",95 , 209, 1)]


# pack variable specify the items and the capacity for the knapsack the student should carry

pack = solve(num,500)
println("The Student should pack: \n - ", join(pack, "\n - "))
println("\nPacked weight: ", sum(getfield.(pack, :weight)), " kg")
println("Packed value: ", sum(getfield.(pack, :value)), " €")        